In [1]:
import logging
import os

import geopandas as gpd

import deafrica_waterbodies.attributes
import deafrica_waterbodies.io
from deafrica_waterbodies.cli.logs import logging_setup

In [2]:
import os

# These are the default AWS configurations for the Analysis Sandbox.
# that are set in the environmnet variables.
aws_default_config = {
    # "AWS_NO_SIGN_REQUEST": "YES",
    "AWS_SECRET_ACCESS_KEY": "fake",
    "AWS_ACCESS_KEY_ID": "fake",
}

# To access public bucket, need to remove the AWS credentials in
# the environment variables or the following error will occur.
# PermissionError: The AWS Access Key Id you provided does not exist in our records.

for key in aws_default_config.keys():
    if key in os.environ:
        del os.environ[key]

In [3]:
verbose = 1
output_directory = "s3://deafrica-waterbodies-dev/test_out_dir/0-0-1/shapefile"
product_version = "0.0.1"
timeseries_bucket = "deafrica-waterbodies-dev"

In [4]:
# Set up logger.
logging_setup(verbose=verbose)
_log = logging.getLogger(__name__)

In [5]:
# Support pathlib paths.
output_directory = str(output_directory)

In [6]:
_log.info("Loading filtered waterbody polygons...")
filtered_polygons_fp = os.path.join(output_directory, "filtered_polygons.parquet")
filtered_polygons = gpd.read_parquet(filtered_polygons_fp)
_log.info(f"Waterbody polygons count {len(filtered_polygons)}.")

[2023-10-05 18:30:23,795] {1292984903.py:1} INFO - Loading filtered waterbody polygons...
[2023-10-05 18:30:24,216] {1292984903.py:4} INFO - Waterbody polygons count 11700.


In [7]:
waterbodies_gdf = deafrica_waterbodies.attributes.assign_unique_ids(polygons=filtered_polygons)
waterbodies_gdf = deafrica_waterbodies.attributes.add_area_and_perimeter_attributes(polygons=waterbodies_gdf)
waterbodies_gdf = deafrica_waterbodies.attributes.add_timeseries_attribute(
    polygons=waterbodies_gdf, product_version=product_version, timeseries_bucket=timeseries_bucket
)

In [8]:
# Reproject to EPSG:4326
waterbodies_gdf_4326 = waterbodies_gdf.to_crs("EPSG:4326")

In [9]:
# Write to disk.
deafrica_waterbodies.io.write_waterbodies_to_file(
    waterbodies_gdf=waterbodies_gdf_4326, product_version=product_version, output_directory=output_directory
)

[2023-10-05 18:30:26,419] {collection.py:558} WARNING - Value 208616399.944799989 of field area_m2 of feature 2521 not successfully written. Possibly due to too larger number with respect to field width
[2023-10-05 18:30:26,930] {collection.py:558} WARNING - Value 110891699.963799998 of field area_m2 of feature 5155 not successfully written. Possibly due to too larger number with respect to field width
[2023-10-05 18:30:27,288] {collection.py:558} WARNING - Value 155855699.958400011 of field area_m2 of feature 7017 not successfully written. Possibly due to too larger number with respect to field width
[2023-10-05 18:30:27,315] {collection.py:558} WARNING - Value 174557699.953099996 of field area_m2 of feature 7122 not successfully written. Possibly due to too larger number with respect to field width
[2023-10-05 18:30:27,589] {collection.py:558} WARNING - Value 190799407.199099988 of field area_m2 of feature 8401 not successfully written. Possibly due to too larger number with respect 